# ライブラリ，ユーティリティ

## インストール

・Linux

In [ ]:
"""
$ pip install git+https://github.com/BindsNET/bindsnet.git
"""

・Google Colab

In [ ]:
"""
!pip install git+https://github.com/BindsNET/bindsnet.git
"""

## インポート

In [ ]:
#-------------------------------------------------
# インポート
#-------------------------------------------------

from abc import ABCMeta, abstractmethod
from time import time as t
import warnings
import random
import sys

import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#import tensorflow_datasets as tfds                # データセット（MNIST, Iris）
from torchvision.datasets import MNIST            # データセット（MNIST）
from sklearn.datasets import load_digits          # データセット（digits）
from sklearn.datasets import load_iris            # データセット（Iris）

from bindsnet.models import DiehlAndCook2015      # Diehl&Cook(2015)モデル
from bindsnet.evaluation import all_activity, assign_labels
from bindsnet.network.monitors import Monitor


## 環境設定

In [ ]:
#-------------------------------------------------
# 環境設定
#-------------------------------------------------

warnings.filterwarnings("ignore", category=DeprecationWarning)
np.set_printoptions(formatter={"float":"{:7.4f}".format})   # 数値の表示を整える
%config InlineBackend.figure_format = "retina"              # 図の描画を高精細に


## ユーティリティ

In [ ]:
#-------------------------------------------------
# ユーティリティ
#-------------------------------------------------

def reset_seed(seed = 3407):                      # 乱数シードをリセット
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

def shuffle(data, label):                         # 要素の対応関係を保ったままシャッフル
    zipped = list(zip(data, label))
    np.random.shuffle(zipped)
    tupple = list(zip(*zipped))
    return (np.array(tupple[0]), np.array(tupple[1]))

def vector_to_matrix(vector):                     # ベクトルを強制的に行列に変換
    length = len(vector)                          # ベクトルの長さを取得
    cols = int(np.ceil(np.sqrt(length)))          # 列のサイズを決定
    rows = cols - 1 if (cols - 1) * cols >= length else cols   # 行を列よりも1つ少なくしても入り切るか？
    matrix = np.full((rows, cols), None)          # Noneで埋められた新しい行列を作成
    matrix.flat[:length] = vector
    return matrix

def judge(flag):                                  # 論理値を判定文字列に変換
    return "✓ PASS" if flag else "✗ FAIL"


# データセットを内部表現に変換

## スケーラ（Scaler）

In [ ]:
#-------------------------------------------------
# スケーラ
#-------------------------------------------------

class Scaler():
    def __init__(self, axis=None):
        self.axis = axis

    @staticmethod
    def passthrough_scaler(data, axis):           # パススルー
        return data

    @staticmethod
    def z_score_scaler(data, axis):               # z-score正規化
        mean = data.mean(axis=axis, keepdims=True)
        std  = data.std (axis=axis, keepdims=True)
        return (data - mean) / std

    @staticmethod
    def min_max_scaler(data, axis):               # Min-Max正規化
        min = data.min(axis=axis, keepdims=True)
        max = data.max(axis=axis, keepdims=True)
        return (data - min) / (max - min)

    def passthrough(self, data):                  # パススルー
        return self.passthrough_scaler(data, self.axis)

    def z_score(self, data):                      # z-score正規化
        return self.z_score_scaler(data, self.axis)

    def min_max(self, data):                      # Min-Max正規化
        return self.min_max_scaler(data, self.axis)


## ニューラルエンコーダ（Neural Encoder）

In [ ]:
#-------------------------------------------------
# ニューラルエンコーダ
#-------------------------------------------------

class NeuralEncoder():
    def __init__(self, Hypr):
        self.lamb     = Hypr.lamb
        self.duration = Hypr.duration
        self.dt       = Hypr.dt

    @staticmethod
    def poisson_simple_encoder(scaled, lamb, duration, dt):     # ポアソンエンコーダ（簡易版）
        steps  = int(duration / dt)
        scaled = scaled[:, np.newaxis]
        lamb   = scaled * lamb * dt
        dist   = np.random.poisson(lamb, (scaled.size, steps))
        return (dist > 0).astype(int)

    @staticmethod
    def poisson_formal_encoder(scaled, lamb, duration, dt):     # ポアソンエンコーダ
        pass                                                    # 未実装

    def poisson_simple(self, scaled):                           # ポアソンエンコーダ（簡易版）
        return self.poisson_simple_encoder(scaled, self.lamb, self.duration, self.dt)

    def poisson_formal(self, scaled):                           # ポアソンエンコーダ
        return self.poisson_formal_encoder(scaled, self.lamb, self.duration, self.dt)


## サブセット（Subset）

In [ ]:
#-------------------------------------------------
# サブセットの基底クラス
#-------------------------------------------------

class SubsetBase(metaclass=ABCMeta):
    def __init__(self, scaler):
        self.scaler = scaler                      # 生データの正規化に使用するスケーラ
        self.data   = None                        # 生データ
        self.scaled = None                        # 正規化されたデータ
        self.label  = None                        # 正解ラベル

    def __call__(self, data, label, i, j):        # サブセットを構築
        self.data  = data [i:j]
        self.label = label[i:j]
        self.scaling_coding()

    @abstractmethod
    def scaling_coding(self):                     # スケーラとエンコーダを適用
        pass

#-------------------------------------------------
# ANNサブセット
#-------------------------------------------------

class AnnSubset(SubsetBase):
    def __init__(self, scaler, classes):
        super().__init__(scaler)
        self.onehot  = None                       # One-Hotラベル
        self.classes = classes

    @staticmethod
    def onehot_encoder(label, classes):           # One-Hotエンコーダ
        onehot = np.zeros((label.size, classes))
        onehot[np.arange(label.size), label] = 1.0
        return onehot

    def scaling_coding(self):
        self.scaled = self.scaler(self.data)
        self.onehot = self.onehot_encoder(self.label, self.classes)

#-------------------------------------------------
# SNNサブセット
#-------------------------------------------------

class SnnSubset(SubsetBase):
    def __init__(self, scaler, encoder):
        super().__init__(scaler)
        self.encoder = encoder                    # 選択されたエンコーダ
        self.spiket  = None                       # スパイクトレイン

    def scaling_coding(self):
        self.scaled = self.scaler(self.data)
        self.spiket = np.array([self.encoder(x).T for x in self.scaled.copy()])


## スーパーセット（Superset）

In [ ]:
#-------------------------------------------------
# スーパーセットの基底クラス
#-------------------------------------------------

class SupersetBase(metaclass=ABCMeta):
    def load(self, dataset, extract):
        (data, label) = dataset(extract)          # データセットを読み込む
        (data, label) = shuffle(data, label)      # データセット全体をシャッフル
        (ex0, ex1, ex2) = extract                 # 抽出サイズ
        self.train(data, label, 0, ex0)
        self.valid(data, label, ex0, ex0+ex1)
        self.test (data, label, ex0+ex1, ex0+ex1+ex2)

#-------------------------------------------------
# ANNスーパーセット
#-------------------------------------------------

class AnnSuperset(SupersetBase):
    def __init__(self, scaler, classes):
        self.train = AnnSubset(scaler, classes)   # トレーニングセット
        self.valid = AnnSubset(scaler, classes)   # バリデーションセット
        self.test  = AnnSubset(scaler, classes)   # テストセット

#-------------------------------------------------
# SNNスーパーセット
#-------------------------------------------------

class SnnSuperset(SupersetBase):
    def __init__(self, scaler, encoder):
        self.train = SnnSubset(scaler, encoder)   # トレーニングセット
        self.valid = SnnSubset(scaler, encoder)   # バリデーションセット
        self.test  = SnnSubset(scaler, encoder)   # テストセット


## データセット（Dataset）

In [ ]:
#-------------------------------------------------
# データセットの基底クラス
#-------------------------------------------------

class DatasetBase(metaclass=ABCMeta):
    @staticmethod
    def bulk_load(dataset, count):                # 指定された件数のデータを一括で読み込む
        data, label = [], []                      #（tensorflow_datasets専用）
        dataset = tfds.as_numpy(dataset.take(count))
        for (dt, lb) in dataset:
            data.append(dt)
            label.append(lb)
        return (np.array(data), np.array(label))

#-------------------------------------------------
# MNISTデータセット
#-------------------------------------------------

class DatasetMNIST(DatasetBase):
    class attr():
        __maxrec = 60000                          # 最大60,000件
        inputs   = 784                            # 画素数（28×28）
        shape    = (1, 28, 28)
        classes  = 10                             # 0-9
    """
    def __call__(self, extract):                  # データセットを読み込む（tensorflow_datasetsから）
        dataset = tfds.load("mnist", as_supervised=True)
        (data, label) = self.bulk_load(dataset["train"], sum(extract))
        data = data.reshape(-1, self.attr.inputs)
        return (data, label)
    """
    def __call__(self, extract):                  # データセットを読み込む（torchvision.datasetsから）
        dataset = MNIST(root="./dataset", train=True, download=True)
        data    = dataset.data.numpy().reshape(-1, self.attr.inputs)
        label   = dataset.targets.numpy()
        return (data, label)

#-------------------------------------------------
# digitsデータセット
#-------------------------------------------------

class DatasetDigits(DatasetBase):
    class attr():
        __maxrec = 1797                           # 最大1,797件
        inputs   = 64                             # 画素数（8×8）
        shape    = (1, 8, 8)
        classes  = 10                             # 0-9

    def __call__(self, extract):                  # データセットを読み込む（sklearn.datasetsから）
        dataset = load_digits()
        return (dataset.data, dataset.target)

#-------------------------------------------------
# Irisデータセット
#-------------------------------------------------

class DatasetIris(DatasetBase):
    class attr():
        __maxrec = 150                            # 最大150件
        inputs   = 4                              # 4つの特徴量
        shape    = (1, 4)
        classes  = 3                              # 3品種
    """
    def __call__(self, extract):                  # データセットを読み込む（tensorflow_datasetsから）
        dataset = tfds.load("iris", as_supervised=True)
        (data, label) = self.bulk_load(dataset["train"], sum(extract))
        return (data, label)
    """
    def __call__(self, extract):                  # データセットを読み込む（sklearn.datasetsから）
        dataset = load_iris()
        return (dataset.data, dataset.target)


# 観察ツール

## サブセットビューア（Subset Viewer）

In [ ]:
#-------------------------------------------------
# サブセットビューア
#-------------------------------------------------

class SubsetViewer():
    def __init__(self, subset, shape):
        self.data   = subset.data
        self.scaled = subset.scaled
        self.label  = subset.label
        self.shape  = (-1,) + shape[1:]

    def show_raw_data(self, i, j):                # 原本の特徴量データを表示
        print(self.data[i:j].reshape(self.shape))

    def show_raw_scaled(self, i, j):              # スケーリング済みの特徴量データを表示
        print(self.scaled[i:j].reshape(self.shape))

    def show_img_data(self, i, j, fs=(6,4)):      # 特徴量データをグレー画像として表示
        data  = self.data[i:j].reshape(self.shape)
        label = self.label[i:j]
        plt.figure(figsize=fs)
        plt.subplots_adjust(wspace=0, hspace=0)
        for (x, (data, label)) in enumerate(zip(data, label)):
            plt.subplot(2, 5, x + 1)
            plt.axis("off")
            plt.title(label)
            plt.imshow(data, vmin=data.min(), vmax=data.max(), cmap="Greys")
        plt.show()


## スパイクビューア（Spike Viewer）

In [ ]:
#-------------------------------------------------
# スパイクビューア
#-------------------------------------------------

class SpikeViewer():
    def __init__(self, spiket):
        self.spiket = spiket.T

    def show_bit_pattern(self):                   # ビットパターンを表示
        display(pd.DataFrame(self.spiket))

    def show_raster_plot(self, fig=(8,4)):        # ラスタープロットを表示
        plt.figure(figsize=fig)
        x_axis = range(self.spiket.shape[1])
        raster = np.where(self.spiket == 0, np.nan, self.spiket) # 0をNaNに置換
        for (i, raster) in enumerate(raster):
            plt.scatter(x_axis, raster*i, s=1.0, c="mediumblue")
        yticks = plt.gca().get_yticks().astype(int)
        plt.yticks(yticks, yticks)
        plt.xlabel("Step #")
        plt.ylabel("Neuron #")
        plt.show()

    def show_spike_train(self, i, j, fs=(8,4)):   # スパイクトレインを表示
        plt.figure(figsize=fs)
        plt.grid(True, linestyle="--", linewidth=0.5)
        event = [np.where(_)[0] for _ in self.spiket[i:j]]  # スパイクイベントを収集
        plt.eventplot(event, linelengths=0.5, colors="mediumblue")
        yticks = plt.gca().get_yticks().astype(int)
        plt.yticks(yticks, yticks + i)
        plt.xlabel("Step #")
        plt.ylabel("Neuron #")
        plt.show()


# 評価指標と性能評価

## メトリクス（Metrics）

In [ ]:
#-------------------------------------------------
# ANNメトリクス
#-------------------------------------------------

class AnnMetrics():
    def __init__(self, Fn_loss):
        self.Fn_loss = Fn_loss                    # 損失関数
        self.loss = []                            # 損失
        self.accu = []                            # 正解率

    def append(self, pred, onehot):               # 測定結果を追加（1エポックごと）
        self.loss.append(self.Fn_loss(pred, onehot))
        self.accu.append(ann_accuracy(pred, onehot))

#-------------------------------------------------
# SNNメトリクス
#-------------------------------------------------

class SnnMetrics():
    def __init__(self):
        self.accu  = []                           # 正解率

    def append(self, pred, label):                # 測定結果を追加（1エポックごと）
        self.accu.append(snn_accuracy(pred, label))


## メジャー（Measure）

In [ ]:
#-------------------------------------------------
# メジャーの基底クラス
#-------------------------------------------------

class MeasureBase(metaclass=ABCMeta):
    def init(self, owner, epochs, verbose):
        self.owner   = owner
        self.epochs  = epochs
        self.verbose = verbose

    @abstractmethod
    def plot_metrics(self):                       # 評価指標をプロット
        pass

    @abstractmethod
    def show_metrics(self):                       # 評価指標を表示
        pass

    def show_each_epoch(self, step, time):
        print(f"epoch  = {step+1:>3d} / {self.epochs:>3d}, "
              f"train accu = {self.train.accu[-1]:.3f}, "
              f"valid accu = {self.valid.accu[-1]:.3f}, "
              f"time = {time:.1f} sec（{round(time/60)} min）")

#-------------------------------------------------
# ANNメジャー
#-------------------------------------------------

class AnnMeasure(MeasureBase):
    def __init__(self, Fn_loss):
        self.train = AnnMetrics(Fn_loss)          # 測定結果（トレーニングセット）
        self.valid = AnnMetrics(Fn_loss)          # 測定結果（バリデーションセット）

    def __call__(self, epoch, time):
        subset = self.owner.superset.train        # トレーニングセットを測定
        pred = self.owner.predict(subset.scaled)
        self.train.append(pred, subset.onehot)
        subset = self.owner.superset.valid        # バリデーションセットを測定
        pred = self.owner.predict(subset.scaled)
        self.valid.append(pred, subset.onehot)
        if self.verbose: self.show_each_epoch(epoch, time)

    def plot_metrics(self):
        plt.ylim(0.0, 1.2)
        plt.grid()
        plt.plot(self.train.accu, label="train accu")
        plt.plot(self.valid.accu, label="valid accu")
        plt.plot(self.train.loss, label="train loss")
        plt.plot(self.valid.loss, label="valid loss")
        plt.xlabel("epochs")
        plt.legend()
        plt.show()

    def show_metrics(self):
        print("Metrics：")
        print(f"  train accu = {self.train.accu[-1]:.3f}")
        print(f"  valid accu = {self.valid.accu[-1]:.3f}")
        print(f"  train loss = {self.train.loss[-1]:.3f}")
        print(f"  valid loss = {self.valid.loss[-1]:.3f}")

#-------------------------------------------------
# SNNメジャー
#-------------------------------------------------

class SnnMeasure(MeasureBase):
    def __init__(self):
        self.train = SnnMetrics()                 # 測定結果（トレーニングセット）
        self.valid = SnnMetrics()                 # 測定結果（バリデーションセット）

    def __call__(self, epoch, time):
        limit = 50                                # SNNは遅いので測定件数を50件に制限
        subset = self.owner.superset.train        # トレーニングセットを測定
        pred = self.owner.predict(subset.spiket[0:limit])
        self.train.append(pred, subset.label[0:limit])
        subset = self.owner.superset.valid        # バリデーションセットを測定
        pred = self.owner.predict(subset.spiket[0:limit])
        self.valid.append(pred, subset.label[0:limit])
        if self.verbose: self.show_each_epoch(epoch, time)

    def plot_metrics(self):
        plt.ylim(0.0, 1.2)
        plt.grid()
        plt.plot(self.train.accu, label="train accu")
        plt.plot(self.valid.accu, label="valid accu")
        plt.xlabel("epochs")
        plt.legend()
        plt.show()

    def show_metrics(self):
        print("Metrics：")
        print(f"  train accu = {self.train.accu[-1]:.3f}")
        print(f"  valid accu = {self.valid.accu[-1]:.3f}")


## 暫定正解率（Provisional Accuracy）

In [ ]:
#-------------------------------------------------
# 暫定正解率
#-------------------------------------------------

class Provaccu():
    def __init__(self, epochs, total):
        self.epochs = epochs
        self.total = total
        self.reset(None)

    def reset(self, epoch):                       # 暫定正解率の算出用データを初期化
        self.epoch = epoch
        self.correctans = 0                       # 暫定正解数の累計
        self.cumulative = 0                       # 暫定データ数の累計

    def update(self, pred, labels):               # 暫定正解率の算出用データを更新
        self.correctans += np.sum(pred == labels)
        self.cumulative += len(labels)

    def show_prov_accu(self, step, time):         # 暫定正解率を表示
        accu = round(self.correctans / self.cumulative, 4)
        print(f"update = {step + 1:>3d} / {self.total}, "
              f"accu = {accu:.3f}, "
              f"time = {time:.1f} sec（{round(time/60)} min）")


## パフォーマンス（Performance）

In [ ]:
#-------------------------------------------------
# パフォーマンスの基底クラス
#-------------------------------------------------

class PerformBase(metaclass=ABCMeta):
    def __init__(self, owner):
        self.owner = owner

    @staticmethod
    def show_judge(step, pred, label):            # 判定結果を表示
        matched = pred == label
        print(f"data = {step:>2d}, "
              f"pred = {pred}, label = {label}, "
              f"judge = {judge(matched)}")

    @staticmethod
    def show_accuracy(accu):                      # 正解率を表示
        print(f"test accu = {accu:.3f}")

#-------------------------------------------------
# ANNパフォーマンス
#-------------------------------------------------

class AnnPerform(PerformBase):
    def __call__(self, subset):                   # 判定結果と正解率を表示
        pred = self.owner.predict(subset.scaled)
        for step, (p, o) in enumerate(zip(pred, subset.onehot)):
            self.show_judge(step, np.argmax(p), np.argmax(o))
        self.show_accuracy(ann_accuracy(pred, subset.onehot))

#-------------------------------------------------
# SNNパフォーマンス
#-------------------------------------------------

class SnnPerform(PerformBase):
    def __call__(self, subset):                   # 判定結果と正解率を表示
        pred = self.owner.predict(subset.spiket)
        for step, (p, l) in enumerate(zip(pred, subset.label)):
            self.show_judge(step, p, l)
        self.show_accuracy(snn_accuracy(pred, subset.label))


# ニューラルネットワーク

## 関数群

In [ ]:
#-------------------------------------------------
# ANN初期化関数（initialization function）        → Fn_init
#-------------------------------------------------

def glorot_uniform(rows, cols):                   # glorot uniform
    min = -np.sqrt(6 / (rows + cols))
    max =  np.sqrt(6 / (rows + cols))
    return np.random.uniform(min, max, (rows, cols))

def he_normal(rows, cols):                        # he normal (for ReLU)
    std = np.sqrt(2 / cols)
    return np.random.normal(0.0, std, (rows, cols))

def xavier_normal(rows, cols):                    # xavier normal (for sigmoid, tanh)
    std = 1 / np.sqrt(cols)
    return np.random.normal(0.0, std, (rows, cols))

#-------------------------------------------------
# ANN活性化関数（activation function）            → Fn_actv
#-------------------------------------------------

def sigmoid(x):                                   # sigmoid関数
    return 1 / (1 + np.exp(-x))

def tanh(x):                                      # tanh関数
    return np.tanh(x)

def relu(x):                                      # ReLU関数
    return np.maximum(0, x)

def softmax(x):                                   # softmax関数
    axis = 0 if (x.ndim == 1) else 1                    # 0：ベクトル対応，1：行列対応
    exps = np.exp(x - x.max(axis=axis, keepdims=True))  # オーバーフロー対策
    return exps / exps.sum(axis=axis, keepdims=True)

#-------------------------------------------------
# ANN導関数（derivative function）                → Fn_drvt
#-------------------------------------------------

def sigmoid_der(x):                               # sigmoid関数の導関数
    return sigmoid(x) * (1.0 - sigmoid(x))

def tanh_der(x):                                  # tanh関数の導関数
    return 1.0 - (tanh(x) ** 2)

def relu_der(x):                                  # ReLU関数の導関数
    return np.where(x > 0, 1, 0)

#-------------------------------------------------
# ANN損失関数（loss function）                    → Fn_loss
#-------------------------------------------------
def crossentropy(y, t):                           # 交差エントロピー損失関数
    y = np.clip(y, 1e-15, 1 - 1e-15)
    return -np.sum(t * np.log(y), axis=1).mean()

#-------------------------------------------------
# 評価関数
#-------------------------------------------------

def ann_accuracy(y, t):                           # ANN正解率関数
    return np.mean(np.argmax(y, axis=1) == np.argmax(t, axis=1))

def snn_accuracy(y, t):                           # SNN正解率関数
    return np.mean(y == t)


## ANNニューロン（ANN Neuron）

In [ ]:
#-------------------------------------------------
# ANNニューロン
#-------------------------------------------------

class AnnNeuron():
    def __init__(self, W, b):
        self.W = W                                # 重み（ベクトル）
        self.b = b                                # バイアス（1個）
        self.reset()

    def reset(self):                              # ミニバッチの開始前にリセットが必要
        self.W_grad = []                          # 重みの勾配の蓄積用
        self.b_grad = []                          # バイアスの勾配の蓄積用

    def weighted_sum(self, X):                    # 総入力関数
        return X @ self.W + self.b                # 式 (5.1) 総入力値

    def calc_grad(self, X, delta):
        self.W_grad.append(X * delta)             # 式 (5.6) 重みの勾配
        self.b_grad.append(delta)                 # 式 (5.7) バイアスの勾配
        return self.W * delta                     # 式 (5.5) 入力の勾配（レイヤで結線どおりに合算する）

    def update(self, eta):                        # パラメータを更新
        self.W -= np.sum(self.W_grad, axis=0) * eta  # 式 (5.8) 重みの更新
        self.b -= np.sum(self.b_grad, axis=0) * eta  # 式 (5.9) バイアスを更新
        self.reset()                              # 次のミニバッチに備える


## ANNレイヤ（ANN Layer）

In [ ]:
#-------------------------------------------------
# ANNレイヤの基底クラス
#-------------------------------------------------

class AnnLayerBase(metaclass=ABCMeta):
    def __init__(self, funcs):
        (self.Fn_init, self.Fn_actv, self.Fn_drvt) = funcs

    def calc_delta(self, X_grad):
        return X_grad * self.Fn_drvt(self.U)      # 式 (5.4),(7.4)  誤差の勾配（中間層）

    @abstractmethod
    def init(self, rows, cols):                   # パラメータの初期化
        pass

    @abstractmethod
    def fire(self, X):                            # レイヤからの出力を計算
        pass

    @abstractmethod
    def calc_grad(self, delta):                   # 入力の勾配を計算
        pass

    @abstractmethod
    def update(self, eta):                        # パラメータを更新
        pass

#-------------------------------------------------
# ANNレイヤ（ニューロン版）
#-------------------------------------------------

class AnnLayerNeuron(AnnLayerBase):
    def __init__(self, rows, cols, funcs):
        super().__init__(funcs)
        self.neurons = []                         # ニューロンの一覧
        self.init(rows, cols)

    def init(self, rows, cols):                   # ニューロンの初期化と一覧への追加
        for weight in self.Fn_init(rows+1, cols).T:
            neuron = AnnNeuron(weight[0:rows], weight[-1])
            self.neurons.append(neuron)

    def fire(self, X):
        U = [neuron.weighted_sum(X) for neuron in self.neurons]
        self.X = X                                # レイヤへの入力を保存（逆伝播で使用）
        self.U = np.array(U)                      # ニューロンからの出力（逆伝播で使用）
        return self.Fn_actv(self.U)               # 式 (5.2) 活性化値

    def calc_grad(self, delta):
        X_grad = []                               # 入力の勾配を蓄積
        for (neuron, delta) in zip(self.neurons, delta):
            X_grad.append(neuron.calc_grad(self.X, delta))
        return np.sum(X_grad, axis=0)             # 蓄積した入力の勾配をリンクの結線どおりに合算

    def update(self, eta):
        for neuron in self.neurons:
            neuron.update(eta)

#-------------------------------------------------
# ANNレイヤ（行列版）
#-------------------------------------------------

class AnnLayerMatrix(AnnLayerBase):
    def __init__(self, rows, cols, funcs):
        super().__init__(funcs)
        self.init(rows, cols)

    def init(self, rows, cols):
        weight = self.Fn_init(rows+1, cols)
        (self.W, self.b) = (weight[0:rows], weight[-1])

    def fire(self, X):
        self.X = X
        self.U = X @ self.W + self.b              # 式 (7.1) 総入力値
        return self.Fn_actv(self.U)               # 式 (7.2) 活性化値

    def calc_grad(self, delta):
        self.W_grad = self.X.T @ delta            # 式 (7.6) 重みの勾配
        self.b_grad = np.sum(delta, axis=0)       # 式 (7.7) バイアスの勾配
        return delta @ self.W.T                   # 式 (7.5) 入力の勾配

    def update(self, eta):
        self.W -= eta * self.W_grad               # 式 (7.8) 重みの更新
        self.b -= eta * self.b_grad               # 式 (7.9) バイアスの更新


## ANNネットワーク（ANN Network）

In [ ]:
#-------------------------------------------------
# ANNネットワークの基底クラス
#-------------------------------------------------

class AnnNetworkBase(metaclass=ABCMeta):
    def __init__(self, Layer, cols, eta):
        self.Layer = Layer                        # レイヤ・クラスを確定
        self.rows = cols                          # 最初のレイヤの行数を確定
        self.eta = eta                            # 学習率
        self.layers = []                          # レイヤの一覧

    def add(self, cols, funcs):                   # レイヤを追加
        layer = self.Layer(self.rows, cols, funcs)
        self.layers.append(layer)
        self.rows = cols                          # 次のレイヤの行数を確定

    def forward(self, X):                         # 順伝播
        for layer in self.layers:
            X = layer.fire(X)
        return X

    def backward(self, delta):                    # 逆伝播
        layers = list(reversed(self.layers))
        X_grad = layers[0].calc_grad(delta)
        for layer in layers[1:]:
            delta  = layer.calc_delta(X_grad)
            X_grad = layer.calc_grad(delta)

    def update(self):                             # パラメータを更新
        for layer in self.layers:
            layer.update(self.eta)

    @abstractmethod
    def batch(self, ss, mini):
        pass

#-------------------------------------------------
# ANNネットワーク（ニューロン版）
#-------------------------------------------------

class AnnNetworkNeuron(AnnNetworkBase):
    def __init__(self, cols, eta):
        super().__init__(AnnLayerNeuron, cols, eta)

    def forward(self, X):
        Y = []
        for x in X:                               # 行列から1行ずつ取り出して処理する
            Y.append(super().forward(x))
        return np.array(Y)

    def batch(self, ss, mini):
        for i in mini:                            # 行列から1行ずつ取り出して処理する
            Y = super().forward(ss.scaled[i])
            self.backward(Y - ss.onehot[i])       # 式 (5.3) 誤差の勾配（出力層）
        self.update()

#-------------------------------------------------
# ANNネットワーク（行列版）
#-------------------------------------------------

class AnnNetworkMatrix(AnnNetworkBase):
    def __init__(self, cols, eta):
        super().__init__(AnnLayerMatrix, cols, eta)

    def batch(self, ss, mini):
        Y = self.forward(ss.scaled[mini])
        self.backward(Y - ss.onehot[mini])        # 式 (7.3) 誤差の勾配（出力層）
        self.update()


## ラベルマップ（Label Map）

In [ ]:
#-------------------------------------------------
# ラベルマップ（Label Map）
#-------------------------------------------------

class LabelMap():
    def __init__(self, neurons):
        self.data = -torch.ones(neurons)          # ラベルマップを保持

    def show_label_map(self):                     # ラベルマップを表示
        labelmap = self.data.numpy()              # torch→numpy変換
        rect = vector_to_matrix(labelmap)         # ラベルマップを矩形に変換
        display(pd.DataFrame(rect).replace({None:"-"}))

    def show_label_count(self, fs=(5,4)):         # ラベルの出現頻度を表示
        labelmap = self.data.numpy()              # torch→numpy変換
        plt.figure(figsize=fs)
        label, counts = np.unique(labelmap, return_counts=True)
        plt.bar(label, counts, align="center")
        for (i, count) in enumerate(counts):
            plt.text(i, count, str(count), ha="center", va="bottom")
        plt.xlabel("Label")
        plt.ylabel("Count")
        plt.xticks(label)
        plt.show()

    def show_pred_rate(self, res):                # 予測レートを表示
        labelmap = self.data.numpy()              # torch→numpy変換
        print("\nPredicted rate:")
        for label in range(10):
            point = np.sum(res[labelmap == label])  # labelの獲得ポイントを集計
            count = np.sum(labelmap == label)       # labelの出現頻度をカウント
            rate  = point / count                   # 予測レートをを計算
            if rate > 0:
                print(f"label = {label}, point = {point}, "
                      f"count = {count}, rate = {rate:.2f}")


## SNNネットワーク（SNN Network）

In [ ]:
#-------------------------------------------------
# SNNネットワーク（BindsNET版）
#-------------------------------------------------

class SnnNetwork(DiehlAndCook2015):
    def __init__(self, Dsat, Hypr):
        super().__init__(
            n_inpt     = Dsat.inputs,             # 入力層における入力数
            n_neurons  = Hypr.neurons,            # ニューロンの数（興奮性、抑制性）
            exc        = Hypr.exc,                # シナプスの強度（興奮性層→抑制性層）
            inh        = Hypr.inh,                # シナプスの強度（抑制性層→興奮性層）
            dt         = Hypr.dt,                 # 時間分解能（ミリ秒）
            norm       = Hypr.norm,               # 接続強度の正規化（入力層→興奮性層）（学習に大きく影響する）
            inpt_shape = Dsat.shape)              # 入力データの形状
        self.shape     = (Hypr.simsteps, 1,) + Dsat.shape
        self.classes   = Dsat.classes
        self.spikes    = self.__set_monitor(Hypr.simsteps)
        self.labelmap  = LabelMap(Hypr.neurons)
        self.rates     = torch.zeros((Hypr.neurons, self.classes))
        self.Hypr      = Hypr

    def __set_monitor(self, simsteps):            # 監視・記録ツールを設定
        spikes = {}
        for layer in set(self.layers):
            spikes[layer] = Monitor(self.layers[layer], state_vars = ["s"], time = simsteps)
            self.add_monitor(spikes[layer], name="%s_spikes" % layer)
        return spikes

    def trainmode(self, mode, size):              #
        self.train(mode=mode)
        self.spike_record = torch.zeros((size, self.Hypr.simsteps, self.Hypr.neurons))

    def forward(self, spiket, index):             # 入力層にスパイクトレインを投入する
        X = { "X": torch.tensor(spiket).view(self.shape) }
        self.run(inputs = X, time = self.Hypr.duration)
        self.spike_record[index] = self.spikes["Ae"].get("s").squeeze()

    def predict(self):                            # 予測
        return all_activity(
            spikes      = self.spike_record,
            assignments = self.labelmap.data,
            n_labels    = self.classes)

    def mapping(self, labels):                    # ラベルマップ（ニューロンとラベルの対応表）を更新
        self.labelmap.data, _, self.rates = assign_labels(
            spikes   = self.spike_record,
            labels   = torch.tensor(labels),
            n_labels = self.classes,
            rates    = self.rates)


## モデラー（Modeler）

In [ ]:
#-------------------------------------------------
# モデラーの基底クラス
#-------------------------------------------------

class ModelerBase(metaclass=ABCMeta):
    def __init__(self, network, measure, superset): # モデルの構築
        self.network  = network
        self.measure  = measure
        self.superset = superset

    @staticmethod
    def idx_mini(total_size, batch_size):         # ミニバッチ用のインデックスを作成
        whole = np.random.permutation(total_size)
        idx_slice = range(0, total_size, batch_size)
        return [whole[i:i+batch_size] for i in idx_slice]

    @abstractmethod
    def training(self, Hypr, verbose=True):       # モデルの訓練
        pass

    @abstractmethod
    def evaluate(self):                           # モデルの性能評価
        pass

    @abstractmethod
    def predict(self, processed):                 # 推論（予測）
        pass

#-------------------------------------------------
# ANNモデラー
#-------------------------------------------------

class AnnModeler(ModelerBase):
    def add(self, cols, Fn_init, Fn_actv, Fn_drvt=lambda x:None): # レイヤを追加
        funcs = (Fn_init, Fn_actv, Fn_drvt)
        self.network.add(cols, funcs)

    def training(self, Hypr, verbose=True):       # モデルの訓練
        subset = self.superset.train              # トレーニングセットを選択
        start = t()
        size = len(subset.scaled)
        self.measure.init(self, Hypr.epochs, verbose)
        self.measure(-1, t()-start)
        for epoch in range(Hypr.epochs):
            whole = self.idx_mini(size, Hypr.batchsz)
            for mini in whole:
                self.network.batch(subset, mini)
            self.measure(epoch, t()-start)

    def evaluate(self):                           # モデルの性能評価
        AnnPerform(self)(self.superset.test)      # テストセットを使って性能を評価

    def predict(self, scaled):                    # 推論（予測）
        return self.network.forward(scaled)

#-------------------------------------------------
# SNNモデラー
#-------------------------------------------------

class SnnModeler(ModelerBase):
    def training(self, Hypr, verbose=True):       # モデルの訓練
        subset = self.superset.train              # トレーニングセットを選択
        total = Hypr.extract[0]
        start = t()
        index = range(total)
        self.prov = Provaccu(Hypr.epochs, total)  # 暫定正解率
        self.measure.init(self, Hypr.epochs, verbose)
        self.measure(-1, t()-start)
        for epoch in range(Hypr.epochs):
            self.network.trainmode(mode=True, size=Hypr.batchsz)
            self.prov.reset(epoch)
            labels = []
            whole = iter(zip(subset.spiket, subset.label))
            for step in range(0, total, Hypr.batchsz):
                for i in index[step:step+Hypr.batchsz]:
                    (spiket, label) = next(whole)
                    self.network.forward(spiket, i % Hypr.batchsz)
                    self.network.reset_state_variables()
                    labels.append(label)
                pred = self.network.predict()[0:len(labels)].numpy()
                self.network.mapping(labels)      # Hypr.batchsz回に1回、ラベルマップを更新する
                if verbose:
                    self.prov.update(pred, np.array(labels))
                    self.prov.show_prov_accu(i, t()-start)
                labels = []
            self.measure(epoch, t()-start)

    def __predict(self, spiket):                  # 予測（1件単位）
        self.network.forward(spiket, 0)
        pred = self.network.predict()[0]
        self.network.reset_state_variables()
        return pred

    def predict(self, spiket):                    # 推論（予測）
        self.network.trainmode(mode=False, size=1)
        return np.array([self.__predict(st) for st in spiket])

    def evaluate(self):                           # モデルの性能評価
        SnnPerform(self)(self.superset.test)      # テストセットを使って性能を評価

    def reaction(self, spiket):                   # ニューロンの反応を表示
        self.network.trainmode(mode=False, size=1)
        self.network.forward(spiket, 0)
        self.network.reset_state_variables()
        resmap = self.network.spike_record.sum(1)[0].numpy()
        rect = vector_to_matrix(resmap)                  # 反応マップを矩形に変換
        display(pd.DataFrame(rect).replace({None:"-"}))  # ニューロンの反応を表示
        self.network.labelmap.show_pred_rate(resmap)     # 予測レートを表示


# 体験コーナー

## ANNの訓練と推論を体験する（digits）

In [ ]:
class Hypr():
    extract  = (300, 50, 50)                      # サブセットの抽出サイズ（train, valid, test）
    epochs   = 50                                 # エポック数
    eta      = 0.01                               # 学習率
    batchsz  = 10                                 # バッチサイズ

In [ ]:
reset_seed()                                      # 乱数シードをリセット
dataset = DatasetDigits()                         # digitsデータセットを選択
Dsat = dataset.attr                               # データセットの属性を取得
scaler = Scaler().min_max                         # Min-Max正規化を選択
superset = AnnSuperset(scaler, Dsat.classes)      # ANNスーパーセットを選択
superset.load(dataset, Hypr.extract)              # ANNスーパーセットにデータセットを読み込む

In [ ]:
sample = SubsetViewer(superset.train, Dsat.shape) # 観察対象にトレーニングセットを指定
sample.show_img_data(20, 30)                      # data #20 〜 data #29 のイメージを表示
sample.show_raw_data(29, 30)                      # data #29 を表示
sample.show_raw_scaled(29, 30)                    # scaled #29 を表示

In [ ]:
network = AnnNetworkMatrix(Dsat.inputs, Hypr.eta) # ANNネットワーク（行列版）を選択
measure = AnnMeasure(crossentropy)                # ANNメジャーを選択
model = AnnModeler(network, measure, superset)    # ANNモデルを構築
model.add(32, xavier_normal, tanh, tanh_der)      # 中間層（for digits）
model.add(Dsat.classes, glorot_uniform, softmax)  # 出力層
model.training(Hypr)                              # ANNモデルを訓練

In [ ]:
measure.plot_metrics()                            # 評価指標をプロット
measure.show_metrics()                            # 評価指標を表示

In [ ]:
reset_seed()                                      # 乱数シードをリセット
model.evaluate()                                  # モデルの性能を評価

In [ ]:
sys.exit("done.")

## ANNの訓練と推論を体験する（MNIST）

In [ ]:
class Hypr():
    extract  = (10000, 1000, 100)                 # サブセットの抽出サイズ（train, valid, test）
    epochs   = 30                                 # エポック数
    eta      = 0.01                               # 学習率
    batchsz  = 10                                 # バッチサイズ

In [ ]:
reset_seed()                                      # 乱数シードをリセット
dataset = DatasetMNIST()                          # MNISTデータセットを選択
Dsat = dataset.attr                               # データセットの属性を取得
scaler = Scaler().min_max                         # Min-Max正規化を選択
superset = AnnSuperset(scaler, Dsat.classes)      # ANNスーパーセットを選択
superset.load(dataset, Hypr.extract)              # ANNスーパーセットにデータセットを読み込む

In [ ]:
sample = SubsetViewer(superset.train, Dsat.shape) # 観察対象にトレーニングセットを指定
sample.show_img_data(0, 10)                       # data #0 〜 data #9 のイメージを表示

In [ ]:
network = AnnNetworkMatrix(Dsat.inputs, Hypr.eta) # ANNネットワーク（行列版）を選択
measure = AnnMeasure(crossentropy)                # ANNメジャーを選択
model = AnnModeler(network, measure, superset)    # ANNモデルを構築
model.add(32, xavier_normal, tanh, tanh_der)      # 中間層（for MNIST）
model.add(Dsat.classes, glorot_uniform, softmax)  # 出力層
model.training(Hypr)                              # ANNモデルを訓練

In [ ]:
measure.plot_metrics()                            # 評価指標をプロット
measure.show_metrics()                            # 評価指標を表示

In [ ]:
reset_seed()                                      # 乱数シードをリセット
model.evaluate()                                  # モデルの性能を評価

In [ ]:
sys.exit("done.")

## ANNの訓練と推論を体験する（Iris）

In [ ]:
class Hypr():
    extract  = (110, 20, 20)                      # サブセットの抽出サイズ（train, valid, test）
    epochs   = 50                                 # エポック数
    eta      = 0.01                               # 学習率
    batchsz  = 10                                 # バッチサイズ

In [ ]:
reset_seed()                                      # 乱数シードをリセット
dataset = DatasetIris()                           # Irisデータセットを選択
Dsat = dataset.attr                               # データセットの属性を取得
scaler = Scaler(axis=0).z_score                   # z-score正規化を選択
superset = AnnSuperset(scaler, Dsat.classes)      # ANNスーパーセットを選択
superset.load(dataset, Hypr.extract)              # ANNスーパーセットにデータセットを読み込む

In [ ]:
sample = SubsetViewer(superset.train, Dsat.shape) # 観察対象にトレーニングセットを指定
sample.show_raw_data(0, 20)                       # data   #0~#19 を表示
sample.show_raw_scaled(0, 20)                     # scaled #0~#19 を表示

In [ ]:
network = AnnNetworkMatrix(Dsat.inputs, Hypr.eta) # ANNネットワーク（行列版）を選択
measure = AnnMeasure(crossentropy)                # ANNメジャーを選択
model = AnnModeler(network, measure, superset)    # ANNモデルを構築
model.add(32, he_normal, relu, relu_der)          # 中間層（for Iris）
model.add(Dsat.classes, glorot_uniform, softmax)  # 出力層
model.training(Hypr)                              # ANNモデルを訓練

In [ ]:
measure.plot_metrics()                            # 評価指標をプロット
measure.show_metrics()                            # 評価指標を表示

In [ ]:
reset_seed()                                      # 乱数シードをリセット
model.evaluate()                                  # モデルの性能を評価

In [ ]:
sys.exit("done.")

## SNNの訓練と推論を体験する（digits）

In [ ]:
class Hypr():
    extract  = (300, 50, 50)                      # サブセットの抽出サイズ（train, valid, test）
    epochs   = 10                                 # エポック数
    norm     = 6.4                                # 接続強度の正規化（入力層→興奮性層）（学習に大きく影響する）
    neurons  = 100                                # ニューロンの数
    lamb     = 0.3                                # 基準発火率（λ）
    duration = 250                                # シミュレーション期間（ms）
    dt       = 1                                  # 時間分解能（ms）
    simsteps = int(duration / dt)                 # シミュレーションステップ数
    batchsz  = 5                                  # ラベルマップを更新する前のトレース回数
    exc      = 22.5                               # シナプスの強度（興奮性層→抑制性層）
    inh      = 120                                # シナプスの強度（抑制性層→興奮性層）

In [ ]:
reset_seed()                                      # 乱数シードをリセット
dataset = DatasetDigits()                         # digitsデータセットを選択
Dsat = dataset.attr                               # データセットの属性を取得
scaler = Scaler().min_max                         # Min-Max正規化を選択
encoder = NeuralEncoder(Hypr).poisson_simple      # ポアソンエンコーダ（簡易版）を選択
superset = SnnSuperset(scaler, encoder)           # SNNスーパーセットを選択
superset.load(dataset, Hypr.extract)              # SNNスーパーセットにデータセットを読み込む

In [ ]:
sample = SubsetViewer(superset.train, Dsat.shape) # 観察対象にトレーニングセットを指定
sample.show_img_data(20, 30)                      # data #20 〜 data #29 のイメージを表示
sample.show_raw_data(29, 30)                      # data #29 を表示
sample.show_raw_scaled(29, 30)                    # scaled #29 を表示

In [ ]:
sample = SpikeViewer(superset.train.spiket[29])
sample.show_bit_pattern()                         # ビットパターンを表示
sample.show_raster_plot()                         # ラスタープロットを表示
sample.show_spike_train(0, 8)                     # スパイクトレインを表示

In [ ]:
network = SnnNetwork(Dsat, Hypr)                  # SNNネットワークを選択
measure = SnnMeasure()                            # SNNメジャーを選択
model = SnnModeler(network, measure, superset)    # SNNモデルを構築
model.training(Hypr)                              # SNNモデルを訓練

In [ ]:
map = model.network.labelmap                      # ラベルマップを取得
map.show_label_map()                              # ラベルマップを表示
map.show_label_count()                            # ラベルの出現頻度を表示

In [ ]:
measure.plot_metrics()                            # 評価指標をプロット
measure.show_metrics()                            # 評価指標を表示

In [ ]:
reset_seed()                                      # 乱数シードをリセット
model.evaluate()                                  # モデルの性能を評価

## ニューロンの反応（digits）

In [ ]:
model.reaction(superset.test.spiket[3])

In [ ]:
model.reaction(superset.test.spiket[46])

In [ ]:
sys.exit("done.")

## SNNの訓練と推論を体験する（MNIST）

In [ ]:
class Hypr():
    extract  = (300, 50, 50)                      # サブセットの抽出サイズ（train, valid, test）
    epochs   = 10                                 # エポック数
    norm     = 78.4                               # 接続強度の正規化（入力層→興奮性層）（学習に大きく影響する）
    neurons  = 100                                # ニューロンの数
    lamb     = 0.3                                # 基準発火率（λ）
    duration = 250                                # シミュレーション期間（ms）
    dt       = 1                                  # 時間分解能（ms）
    simsteps = int(duration / dt)                 # シミュレーションステップ数
    batchsz  = 5                                  # ラベルマップを更新する前のトレース回数
    exc      = 22.5                               # シナプスの強度（興奮性層→抑制性層）
    inh      = 120                                # シナプスの強度（抑制性層→興奮性層）

In [ ]:
reset_seed()                                      # 乱数シードをリセット
dataset = DatasetMNIST()                          # MNISTデータセットを選択
Dsat = dataset.attr                               # データセットの属性を取得
scaler = Scaler().min_max                         # Min-Max正規化を選択
encoder = NeuralEncoder(Hypr).poisson_simple      # ポアソンエンコーダ（簡易版）を選択
superset = SnnSuperset(scaler, encoder)           # SNNスーパーセットを選択
superset.load(dataset, Hypr.extract)              # SNNスーパーセットにデータセットを読み込む

In [ ]:
sample = SubsetViewer(superset.train, Dsat.shape) # 観察対象にトレーニングセットを指定
sample.show_img_data(0, 10)                       # data #0 〜 data #9 のイメージを表示

In [ ]:
sample = SpikeViewer(superset.train.spiket[4])
sample.show_bit_pattern()                         # ビットパターンを表示
sample.show_raster_plot(fig=(8,6))                # ラスタープロットを表示
sample.show_spike_train(116, 126)                 # スパイクトレインを表示

In [ ]:
network = SnnNetwork(Dsat, Hypr)                  # SNNネットワークを選択
measure = SnnMeasure()                            # SNNメジャーを選択
model = SnnModeler(network, measure, superset)    # SNNモデルを構築
model.training(Hypr)                              # SNNモデルを訓練

In [ ]:
map = model.network.labelmap                      # ラベルマップを取得
map.show_label_map()                              # ラベルマップを表示
map.show_label_count()                            # ラベルの出現頻度を表示

In [ ]:
measure.plot_metrics()                            # 評価指標をプロット
measure.show_metrics()                            # 評価指標を表示

In [ ]:
reset_seed()                                      # 乱数シードをリセット
model.evaluate()                                  # モデルの性能を評価

In [ ]:
sys.exit("done.")

## SNNの訓練と推論を体験する（Iris）

In [ ]:
class Hypr():
    extract  = (110, 20, 20)                      # サブセットの抽出サイズ（train, valid, test）
    epochs   = 4                                  # エポック数
    norm     = 1.5                                # 接続強度の正規化（入力層→興奮性層）（学習に大きく影響する）
    neurons  = 100                                # ニューロンの数
    lamb     = 0.3                                # 基準発火率（λ）
    duration = 250                                # シミュレーション期間（ms）
    dt       = 1                                  # 時間分解能（ms）
    simsteps = int(duration / dt)                 # シミュレーションステップ数
    batchsz  = 5                                  # ラベルマップを更新する前のトレース回数
    exc      = 22.5                               # シナプスの強度（興奮性層→抑制性層）
    inh      = 120                                # シナプスの強度（抑制性層→興奮性層）

In [ ]:
reset_seed()                                      # 乱数シードをリセット
dataset = DatasetIris()                           # Irisデータセットを選択
Dsat = dataset.attr                               # データセットの属性を取得
scaler = Scaler(axis=0).min_max                   # Min-Max正規化を選択
encoder = NeuralEncoder(Hypr).poisson_simple      # ポアソンエンコーダ（簡易版）を選択
superset = SnnSuperset(scaler, encoder)           # SNNスーパーセットを選択
superset.load(dataset, Hypr.extract)              # SNNスーパーセットにデータセットを読み込む

In [ ]:
sample = SubsetViewer(superset.train, Dsat.shape) # 観察対象にトレーニングセットを指定
sample.show_raw_data(0, 20)                       # data   #0~#19 を表示
sample.show_raw_scaled(0, 20)                     # scaled #0~#19 を表示

In [ ]:
sample = SpikeViewer(superset.train.spiket[0])
sample.show_bit_pattern()                         # ビットパターンを表示
sample.show_raster_plot(fig=(8,2))                # ラスタープロットを表示
sample.show_spike_train(0, 4)                     # スパイクトレインを表示

In [ ]:
network = SnnNetwork(Dsat, Hypr)                  # SNNネットワークを選択
measure = SnnMeasure()                            # SNNメジャーを選択
model = SnnModeler(network, measure, superset)    # SNNモデルを構築
model.training(Hypr)                              # SNNモデルを訓練

In [ ]:
map = model.network.labelmap                      # ラベルマップを取得
map.show_label_map()                              # ラベルマップを表示
map.show_label_count()                            # ラベルの出現頻度を表示

In [ ]:
measure.plot_metrics()                            # 評価指標をプロット
measure.show_metrics()                            # 評価指標を表示

In [ ]:
reset_seed()                                      # 乱数シードをリセット
model.evaluate()                                  # モデルの性能を評価

In [ ]:
sys.exit("done.")